In [1]:
# Load Libraries

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import Prompt
from langchain.chains import LLMChain
from dotenv import load_dotenv

In [2]:
load_dotenv()

False

In [9]:
import json
import requests
from requests.exceptions import HTTPError

try:
    response = requests.get("https://api.visitjeju.net/vsjApi/contents/searchList?apiKey=ycvvnzn57g72hwvc&locale=kr&category=c1")
    response.raise_for_status()
    
    jsonResponse = response.json()
    
    print("Entire JSON response")
    print(jsonResponse)

except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')

except Exception as err:
    print(f'Other error occurred: {err}')

Entire JSON response
{'result': '200', 'resultMessage': 'SUCCESS', 'totalCount': 1229, 'resultCount': 100, 'pageSize': 100, 'pageCount': 13, 'currentPage': 1, 'items': [{'alltag': '문화유적지,4.3', 'contentsid': 'CNTS_000000000022876', 'contentscd': {'value': 'c1', 'label': '관광지', 'refId': 'contentscd>c1'}, 'title': '조천중학원 옛터', 'region1cd': {'value': 'region1', 'label': '제주시', 'refId': 'region>region1'}, 'region2cd': {'value': '15', 'label': '조천', 'refId': 'region1>15'}, 'address': '제주특별자치도 제주시 조천읍 조천리 2347-1', 'roadaddress': '제주특별자치도 제주시 조천읍 신북로 264', 'tag': '문화유적지,4.3,문화관광,제주4.3,역사유적,어트랙션', 'introduction': '1947년 3․1절 시위 및 총파업 이후 미군정과 서청의 탄압을 받았던 조천중학원', 'latitude': 33.538307, 'longitude': 126.640305, 'postcode': '63331', 'phoneno': '--', 'repPhoto': {'descseo': '조천중학원 옛터', 'photoid': {'photoid': 16783, 'imgpath': 'https://api.cdn.visitjeju.net/photomng/imgpath/201804/30/f9c3e840-8440-42be-b926-6edaaf8ac525.jpg', 'thumbnailpath': 'https://api.cdn.visitjeju.net/photomng/thumbnailpath/20180

In [23]:
type(jsonResponse)

print("Print each key-value pair from JSON response")

# for key, value in jsonResponse.items():
#     print(key, ":", value)

# jsonResponse["items"][0]

for i in range(1, len(jsonResponse["items"]), 1):
    print(jsonResponse["items"][i]["title"], "\t", jsonResponse["items"][i]["roadaddress"], "\t", jsonResponse["items"][i]["phoneno"], "\t", jsonResponse["items"][i]["introduction"], "\t", jsonResponse["items"][1]["tag"])

Print each key-value pair from JSON response
타오하우스 	 제주특별자치도 제주시 도두봉6길 9-1 	 0507-1404-0328 	 맛과 향을 만끽하는 핸드드립 커피체험 	 제주시내, 도두, 커피, 핸드드립, 이색체험
멍멍플레이스 	 제주특별자치도 제주시 애월읍 소길남길 244-5 	 0507-1307-4661 	 애월읍 중산간 마을에 위치한 멍멍 플레이스는 애견인을 위한 최고의 숙소로 꼽힌다. 강아지와 편안하게 쉴 수 있는 잠자리는 물론 잔디밭에서 맘껏 뛰어 다닐 수 있는 최적의 환경을 갖추고 있다. 반려견과 제주 여행을 계획한다면 한번 눈여겨 볼 필요가 있다.  	 제주시내, 도두, 커피, 핸드드립, 이색체험
제주농업학교 옛터 	 제주특별자치도 제주시 전농로 100 	 -- 	 4․3이 발발하면서 수많은 제주도민들이 잡혀와 고문과 취조를 당했던 한과 눈물의 장소 	 제주시내, 도두, 커피, 핸드드립, 이색체험
북촌환해장성 	 제주특별자치도 제주시 조천읍 북촌리 	 -- 	 계속된 왜구의 침략으로 불안과 공포에 휩싸였던 그 시대의 삶을 느껴보자. 	 제주시내, 도두, 커피, 핸드드립, 이색체험
산지항 옛터 	 제주특별자치도 제주시 연무정길 28 (건입동)일대 	 -- 	 1950년 예비검속 당시 에비검속자 가운데 일부가 수장당한 곳. 	 제주시내, 도두, 커피, 핸드드립, 이색체험
놀자개 	 제주특별자치도 제주시 오광로 143 	 0507-1336-3083 	 놀자개는 제주시 노형동에 위치한 반려견 놀이터다. 반려견을 위한 유치원과 호텔, 미용실도 운영하는 놀자개는 개별 온돌 난방 시스템을 통해 반려동물의 안락한 잠자리를 마련하였다. 	 제주시내, 도두, 커피, 핸드드립, 이색체험
하례점빵 	 제주특별자치도 서귀포시 남원읍 하례로 272 	 064-767-4545 	 새콤달콤 맛 좋은 상웨빵 요리체험 	 제주시내, 도두, 커피, 핸드드립, 이색체험
오림반 	 제주특별자치도 서귀포시 남원읍 신례리 	 -- 	 4․3 초기 무

In [27]:
import csv

f = open("./docs/jeju_visit.csv", "w", encoding='utf-8')

writer = csv.writer(f, delimiter="\t")

for i in range(1, len(jsonResponse["items"]), 1):
    row = [jsonResponse["items"][i]["title"], jsonResponse["items"][i]["roadaddress"], jsonResponse["items"][i]["phoneno"], jsonResponse["items"][i]["introduction"], jsonResponse["items"][1]["tag"]]
    print(row)
    writer.writerow(row)


f.close()

['타오하우스', '제주특별자치도 제주시 도두봉6길 9-1', '0507-1404-0328', '맛과 향을 만끽하는 핸드드립 커피체험', '제주시내, 도두, 커피, 핸드드립, 이색체험']
['멍멍플레이스', '제주특별자치도 제주시 애월읍 소길남길 244-5', '0507-1307-4661', '애월읍 중산간 마을에 위치한 멍멍 플레이스는 애견인을 위한 최고의 숙소로 꼽힌다. 강아지와 편안하게 쉴 수 있는 잠자리는 물론 잔디밭에서 맘껏 뛰어 다닐 수 있는 최적의 환경을 갖추고 있다. 반려견과 제주 여행을 계획한다면 한번 눈여겨 볼 필요가 있다. ', '제주시내, 도두, 커피, 핸드드립, 이색체험']
['제주농업학교 옛터', '제주특별자치도 제주시 전농로 100', '--', '4․3이 발발하면서 수많은 제주도민들이 잡혀와 고문과 취조를 당했던 한과 눈물의 장소', '제주시내, 도두, 커피, 핸드드립, 이색체험']
['북촌환해장성', '제주특별자치도 제주시 조천읍 북촌리', '--', '계속된 왜구의 침략으로 불안과 공포에 휩싸였던 그 시대의 삶을 느껴보자.', '제주시내, 도두, 커피, 핸드드립, 이색체험']
['산지항 옛터', '제주특별자치도 제주시 연무정길 28 (건입동)일대', '--', '1950년 예비검속 당시 에비검속자 가운데 일부가 수장당한 곳.', '제주시내, 도두, 커피, 핸드드립, 이색체험']
['놀자개', '제주특별자치도 제주시 오광로 143', '0507-1336-3083', '놀자개는 제주시 노형동에 위치한 반려견 놀이터다. 반려견을 위한 유치원과 호텔, 미용실도 운영하는 놀자개는 개별 온돌 난방 시스템을 통해 반려동물의 안락한 잠자리를 마련하였다.', '제주시내, 도두, 커피, 핸드드립, 이색체험']
['하례점빵', '제주특별자치도 서귀포시 남원읍 하례로 272', '064-767-4545', '새콤달콤 맛 좋은 상웨빵 요리체험', '제주시내, 도두, 커피, 핸드드립, 이색체험']
['오림반', '제주특별자치도 서귀포시 남원읍 신례리',